Use pip to install the OpenCV software.

In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


Use pip to install the NumPy software.

In [2]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


Import necessary libraries.

In [3]:
import numpy as np
import cv2

This function accepts a path to a video. It then uses the OpenCV software to randomly collect
50 frames from the video and then finds the median of those 50 frames. This median will be used
for background subtraction in order to detect motion within the video.

In [4]:
def median_background(path):
    #open video from a given path
    video = cv2.VideoCapture(path)
    # use 50 random frames for median
    indexes = video.get(cv2.CAP_PROP_FRAME_COUNT) * np.random.uniform(size=50)
    # an array to hold the frames
    frames = []
    
    for i in indexes:
        video.set(cv2.CAP_PROP_POS_FRAMES, i)
        check, frame = video.read()
        frames.append(frame)
        
    # use numpy median function to calculate median of 50 random frames
    med_frame = np.median(frames, axis=0).astype(np.uint8)
    return med_frame

This is the meat and potatoes of the application. It's the function where all the frame 
differencing occurs. It accepts a path to the required video which it sends to 
median_background to find a median frame. The function then converts this background frame 
to gray scale and uses the difference of this with the grayscale converted frames of the
video to determine if motion is taking place. Since we're dealing with the detection of cars
here it only draws green rectangles around objects with areas above a certain threshold so
as not to draw rectangles around people.

In [5]:
def frameDifferencing(path, numframes, rect_loc):
    video = cv2.VideoCapture(path)

    #height of the frame used to only detect cars on main street
    height = int(video.get(4))

    # get median background
    background = median_background(path)
    # convert to grayscale
    background = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
    count = 0
    num_frames = numframes

    city_center = 0
    curr_rect = []
    prev_rect = []
    while (video.isOpened()):
        check, frame = video.read()
        if check == True:
            count += 1
            # make copy of original frame
            original_frame = frame.copy()
            # convert frame to grayscale
            gray_scale = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            # checks whether frame count has reached a multiple of num_frames 
            if count % num_frames == 0 or count == 1:
                diff_arr = []
            # calculates absolute difference between gray scale and background
            abs_diff = cv2.absdiff(gray_scale, background)
            # converts the difference to binary using thresholding
            check, thres = cv2.threshold(abs_diff, 50, 255, cv2.THRESH_BINARY)
            # dilate frame to increase contour detection
            dil_frame = cv2.dilate(thres, None, iterations=2)
            # append the final result into the `diff_arr`
            diff_arr.append(dil_frame)
            # check if diff_arr has reached the required length
            if len(diff_arr) == num_frames:
                # sum the frames in diff_arr
                sum_num_frames = sum(diff_arr)
                # save contours around the white dilated areas
                contours, hierarchy = cv2.findContours(sum_num_frames, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                for i in range(len(contours)):
                    # move past contours with area less than 500 to
                    # filter out noise
                    if cv2.contourArea(contours[i]) < 500:
                        continue
                    (x, y, w, h) = cv2.boundingRect(contours[i])


                    # draw green rectangles
                    if y > height / 2 and w * h > 6000:
                        cv2.rectangle(original_frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                        curr_rect.append((x, y, w, h))

                # find the minimum length of the previous and current
                # array of rectangles
                min_len = min(len(curr_rect), len(prev_rect))
                for i in range(min_len):
                    if curr_rect[i][0] > 400 and curr_rect[i][0] < rect_loc and curr_rect[i][1] > 350 and curr_rect[i][1] < 400 and curr_rect[i][0] < prev_rect[i][0]:
                        # count number of cars going towards city center
                        # and print current value
                        city_center += 1
                        print(city_center)

                # set previous array of rectangles equal to current array of rectangles
                prev_rect = curr_rect
                # reset current array of rectangles
                curr_rect = []

                cv2.imshow('Detected Objects', original_frame)
                if cv2.waitKey(100) & 0xFF == ord('q'):
                    break
        else:
            break
    video.release()
    cv2.destroyAllWindows() 
    return city_center

Call the frame differencing function for both the recordings from the city traffic cameras. 
Store the traffic data of cars heading to the city center in two separate variables.

In [6]:
city_center1 = frameDifferencing('Exercise1_Files/Traffic_Laramie_1.mp4', 4, 435)
city_center2 = frameDifferencing('Exercise1_Files/Traffic_Laramie_2.mp4', 5, 440)

1
2
3
4
5
6
1
2
3
4


Use pip to install the moviepy package used to extract the lengths of each of the traffic recordings.

In [7]:
pip install moviepy

Note: you may need to restart the kernel to use updated packages.


Import the necessary packages from moviepy and use it to extract and print the video lengths.

In [8]:
from moviepy.editor import VideoFileClip
clip1 = VideoFileClip('Exercise1_Files/Traffic_Laramie_1.mp4')
print(clip1.duration)
clip2 = VideoFileClip('Exercise1_Files/Traffic_Laramie_2.mp4')
print(clip2.duration)

177.92000000000002
105.68


Divide the clip durations by 60 to convert the lengths to minutes instead of seconds.

In [9]:
length1 = clip1.duration / 60
length2 = clip2.duration / 60

Import pandas and use it to store the data retrieved from the videos into a dataframe.
Then print the dataframe for the benefit of the user. 

In [10]:
import pandas as pd
data = [["Traffic_Laramie_1.mp4", city_center1, city_center1 / length1], ["Traffic_Laramie_2.mp4", city_center2, city_center2 / length2]]
pd.DataFrame(data, columns=["", "Total Number of Cars", "Cars per Minute"])

,,Total Number of Cars,Cars per Minute
0,Traffic_Laramie_1.mp4,6,2.023381
1,Traffic_Laramie_2.mp4,4,2.271007
